In [1]:
%load_ext autoreload
%autoreload 2

import sys
if '..' not in sys.path:
    sys.path.append('..')
from notebooks.results import load_results, classification, clusterization, to_latex_table, GremDataFrame
import re

In [2]:
def extract_letters(s: str) -> str:
    return re.findall(r'[a-zA-Z]+', s)[0]

def fix_dataset_name(s: str) -> str:
    return {
        'TweeterCyberbullying': 'TwitterCyberbullying'
    }.get(s, s)

In [3]:
# results_df = load_results('../results')
results_df = GremDataFrame(load_results('../../grembedding_results'))
results_df['dataset'] = results_df['dataset'].apply(fix_dataset_name)
results_df['base_head_model'] = results_df['params_name'].apply(extract_letters)
results_df.head()

,dataset,datacleaner,vectorizer,params_name,accuracy,f1_score,recall,precision,silhouette,davies_bouldin,calinski_harabasz,bcubed_precission,bcubed_recall,bcubed_f1,base_head_model
0,TwitterCyberbullying,DummyDatacleaner,SpacyMorphTagVectorizer,MLP1,0.915423,0.477922,0.50000,0.457711,NaN,NaN,NaN,NaN,NaN,NaN,MLP
1,TwitterCyberbullying,DummyDatacleaner,SpacyMorphTagVectorizer,MLP2,0.915423,0.477922,0.50000,0.457711,NaN,NaN,NaN,NaN,NaN,NaN,MLP
2,TwitterCyberbullying,DummyDatacleaner,SpacyMorphTagVectorizer,MLP3,0.917413,0.541487,0.53312,0.760302,NaN,NaN,NaN,NaN,NaN,NaN,MLP
3,TwitterCyberbullying,DummyDatacleaner,SpacyMorphTagVectorizer,RandomForest1,0.915423,0.477922,0.50000,0.457711,NaN,NaN,NaN,NaN,NaN,NaN,RandomForest
4,TwitterCyberbullying,DummyDatacleaner,SpacyMorphTagVectorizer,KMeans1.0,NaN,NaN,NaN,NaN,0.086742,3.113713,889.408505,0.845114,0.501334,0.629336,KMeans


In [9]:
(
    results_df
    .dataset(['PrusVsSienkiewicz', 'StarWarsFanfic'])
    .data_cleaner('DummyDatacleaner')
    .vectorizer('SpacyMorphTagVectorizer')
    .classification()
)

,dataset,datacleaner,vectorizer,params_name,accuracy,f1_score,recall,precision,silhouette,davies_bouldin,calinski_harabasz,bcubed_precission,bcubed_recall,bcubed_f1,base_head_model
85,PrusVsSienkiewicz,DummyDatacleaner,SpacyMorphTagVectorizer,RandomForest1,0.852688,0.831615,0.832492,0.830762,NaN,NaN,NaN,NaN,NaN,NaN,RandomForest
86,PrusVsSienkiewicz,DummyDatacleaner,SpacyMorphTagVectorizer,MLP1,0.752688,0.727253,0.737683,0.721667,NaN,NaN,NaN,NaN,NaN,NaN,MLP
87,PrusVsSienkiewicz,DummyDatacleaner,SpacyMorphTagVectorizer,MLP2,0.781720,0.761466,0.776675,0.754386,NaN,NaN,NaN,NaN,NaN,NaN,MLP
88,PrusVsSienkiewicz,DummyDatacleaner,SpacyMorphTagVectorizer,MLP3,0.800000,0.777115,0.785746,0.771183,NaN,NaN,NaN,NaN,NaN,NaN,MLP
311,StarWarsFanfic,DummyDatacleaner,SpacyMorphTagVectorizer,RandomForest1,0.902439,0.901502,0.899821,0.907599,NaN,NaN,NaN,NaN,NaN,NaN,RandomForest
313,StarWarsFanfic,DummyDatacleaner,SpacyMorphTagVectorizer,MLP1,0.853659,0.851449,0.849732,0.862709,NaN,NaN,NaN,NaN,NaN,NaN,MLP
314,StarWarsFanfic,DummyDatacleaner,SpacyMorphTagVectorizer,MLP2,0.865854,0.864218,0.862552,0.872549,NaN,NaN,NaN,NaN,NaN,NaN,MLP
315,StarWarsFanfic,DummyDatacleaner,SpacyMorphTagVectorizer,MLP3,0.890244,0.889438,0.888193,0.893116,NaN,NaN,NaN,NaN,NaN,NaN,MLP


In [8]:
prus_vs_sienkiewicz = results_df[results_df['dataset'] == 'PrusVsSienkiewicz']
data = prus_vs_sienkiewicz.loc[prus_vs_sienkiewicz.groupby('vectorizer')['accuracy'].idxmax()]
data = data[['vectorizer', 'accuracy', 'f1_score', 'base_head_model']].sort_values('f1_score', ascending=False)
print(to_latex_table(
    data,
    place_modifiers='H',
    separate_header=True,
    column_names=['vectorizer', 'accuracy', 'f1 score', 'base head model'],
    float_precission=2,
    caption='Accuracy i miara F1 dla różnych sposobów wektoryzacji powieści Prusa i Sienkiewicza',
    label='prus_vs_sien:vectorizer_comparison',
    bold_labels=['f1_score', 'accuracy']
))

\begin{table}[H]
	\centering
	\begin{tabular}{||c|c|c|c||}
		\hline
		vectorizer & accuracy & f1 score & base head model \\
		\hline\hline
		CountVectorizer5000 & \textbf{0.86} & \textbf{0.84} & RandomForest \\
		TfidfVectorizer1000 & 0.85 & \textbf{0.84} & MLP \\
		TfidfVectorizer5000 & 0.85 & \textbf{0.84} & RandomForest \\
		CountVectorizer1000 & 0.85 & \textbf{0.84} & MLP \\
		SpacyMorphTagVectorizer & 0.85 & 0.83 & RandomForest \\
		FullMorphTagVectorizer & 0.80 & 0.78 & RandomForest \\
		BigramMorphTagVectorizer370 & 0.77 & 0.75 & MLP \\
		BigramMorphTagVectorizer100 & 0.77 & 0.74 & MLP \\
		DPEBPVectorizer100Avg & 0.66 & 0.65 & MLP \\
		\hline
	\end{tabular}
	\caption{Accuracy i miara F1 dla różnych sposobów wektoryzacji powieści Prusa i Sienkiewicza}
	\label{tab:prus_vs_sien:vectorizer_comparison}
\end{table}


In [12]:
df = results_df[results_df['dataset'] == 'OldNewspapers']
data = df.loc[df.groupby('vectorizer')['accuracy'].idxmax()]
data = data[['vectorizer', 'accuracy', 'f1_score', 'base_head_model']].sort_values('f1_score', ascending=False)
print(to_latex_table(
    data,
    place_modifiers='H',
    separate_header=True,
    column_names=['vectorizer', 'accuracy', 'f1 score', 'base head model'],
    float_precission=2,
    caption='Accuracy i miara F1 dla różnych sposobów wektoryzacji artykułów z gazet',
    label='old_newspapers:vectorizer_comparison',
))

{}
\begin{table}[H]
	\centering
	\begin{tabular}{||c|c|c|c||}
		\hline
		vectorizer & accuracy & f1 score & base head model \\
		\hline\hline
		\hline
	\end{tabular}
	\caption{Accuracy i miara F1 dla różnych sposobów wektoryzacji artykułów z gazet}
	\label{tab:old_newspapers:vectorizer_comparison}
\end{table}


In [25]:
df = results_df
data = df.loc[df.groupby('dataset')['accuracy'].idxmax()]
data = data[['vectorizer', 'dataset', 'accuracy', 'f1_score', 'base_head_model']].sort_values('f1_score', ascending=False)
print(to_latex_table(
    data,
    place_modifiers='H',
    separate_header=True,
    column_names=['vectorizer', 'dataset', 'accuracy', 'f1 score', 'base head model'],
    float_precission=2,
    caption='Najlepsze wektoryzery dla każdego ze zbioru danych (zwględem accuracy)',
    label='best_vectorizer:classification',
))

\begin{table}[H]
	\centering
	\begin{tabular}{||c|c|c|c|c||}
		\hline
		vectorizer & dataset & accuracy & f1 score & base head model \\
		\hline\hline
		TfidfVectorizer5000 & StarWarsFanfic & 1.00 & 1.00 & MLP \\
		TfidfVectorizer1000 & Classics5Authors35Books & 0.93 & 0.91 & MLP \\
		CountVectorizer5000 & PrusVsSienkiewicz & 0.86 & 0.84 & RandomForest \\
		TfidfVectorizer5000 & WritingStyle & 0.72 & 0.69 & RandomForest \\
		FullMorphTagVectorizer & TwitterCyberbullying & 0.93 & 0.67 & MLP \\
		CountVectorizer5000 & EroticVsOthers & 0.66 & 0.66 & RandomForest \\
		HerbertVectorizer & OldNewspapers & 0.40 & 0.24 & MLP \\
		\hline
	\end{tabular}
	\caption{Najlepsze wektoryzery dla każdego ze zbioru danych (zwględem accuracy)}
	\label{tab:best_vectorizer:classification}
\end{table}
